In [1]:
# 02_AQI_Point_Maps_LocalGuideline.ipynb

import os, glob, warnings
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from shapely.ops import unary_union
from fiona.env import Env

# ---- Paths
DATA_DIR  = r"C:\Users\krish\Desktop\SpatialCARE\DailyGPKG"
PASIG_SHP = r"C:\Users\krish\Desktop\PhD Class\Shapefile\MM\Pasig\Pasig.shp"
OUT_DIR   = r"C:\Users\krish\Desktop\SpatialCARE\Outputs\figures\aqi_points_local"
os.makedirs(OUT_DIR, exist_ok=True)

# ---- CRS & figure
WGS84  = "+proj=longlat +datum=WGS84 +no_defs"
UTM51  = "+proj=utm +zone=51 +datum=WGS84 +units=m +no_defs"
FIG_SIZE, FIG_DPI = (7,7), 150

# ---- Column helpers
PM_CANDS  = ["pm25","PM25","PM_25","PM2_5","PM2.5"]
LON_CANDS = ["longitude","lon","LONG","x"]
LAT_CANDS = ["latitude","lat","LAT","y"]

def pick(cols, cands):
    for c in cands:
        if c in cols:
            return c
    return None

# ---- Local PM2.5 guideline (µg/m³, 24-hr)
# Good: 0–25.0
# Fair: 25.1–35.0
# Unhealthy for sensitive groups: 35.1–45.0
# Very unhealthy: 45.1–55.0
# Acutely unhealthy: 55.1–90.0
# Emergency: >= 91.0
NO_DATA_LABEL = "No data"
NO_DATA_COLOR = "#bdbdbd"

AQI_LABELS = [
    "Good (0–25.0)",
    "Fair (25.1–35.0)",
    "Unhealthy for sensitive (35.1–45.0)",
    "Very unhealthy (45.1–55.0)",
    "Acutely unhealthy (55.1–90.0)",
    "Emergency (≥91)"
]
AQI_COLORS = [
    "#00E400",  # Good
    "#FFFF00",  # Fair
    "#FF7E00",  # Unhealthy SG
    "#FF0000",  # Very unhealthy
    "#8F3F97",  # Acutely unhealthy
    "#7E0023"   # Emergency
]

def classify_pm25(val):
    """Return (label, color) per local guideline; robust to NaN/strings."""
    v = pd.to_numeric(val, errors="coerce")
    if not np.isfinite(v):
        return NO_DATA_LABEL, NO_DATA_COLOR
    v = max(0.0, float(v))  # no negatives
    if v <= 25.0:   return AQI_LABELS[0], AQI_COLORS[0]
    elif v <= 35.0: return AQI_LABELS[1], AQI_COLORS[1]
    elif v <= 45.0: return AQI_LABELS[2], AQI_COLORS[2]
    elif v <= 55.0: return AQI_LABELS[3], AQI_COLORS[3]
    elif v <  91.0: return AQI_LABELS[4], AQI_COLORS[4]   # 55.1–90.999…
    else:           return AQI_LABELS[5], AQI_COLORS[5]   # ≥ 91

# ---- Read Pasig boundary
with Env(SHAPE_RESTORE_SHX="YES"):
    pasig = gpd.read_file(PASIG_SHP)
if pasig.crs is None:
    pasig = pasig.set_crs(WGS84)
pasig_utm = pasig.to_crs(UTM51)

# ---- Iterate daily files
files = sorted(glob.glob(os.path.join(DATA_DIR, "date_2025-*.gpkg")))
if not files:
    raise SystemExit("No daily GPKG files found.")

for f in files:
    base = os.path.splitext(os.path.basename(f))[0]
    date_label = base.replace("date_","")

    g = gpd.read_file(f)
    if g.crs is None:
        g = g.set_crs(WGS84)

    # Ensure geometry exists (build from lon/lat if necessary)
    if ("geometry" not in g) or g.geometry.is_empty.all():
        lon = pick(g.columns, LON_CANDS)
        lat = pick(g.columns, LAT_CANDS)
        if lon and lat:
            g = g.set_geometry(gpd.points_from_xy(g[lon], g[lat], crs=g.crs))
        else:
            print(f"Skip (no geometry & no lon/lat): {base}")
            continue

    pm_col = pick(g.columns, PM_CANDS)
    if pm_col is None:
        print(f"Skip (no PM2.5 column): {base}")
        continue

    # Reproject & classify per guideline
    pts = g.to_crs(UTM51).copy()
    classes = pts[pm_col].apply(classify_pm25)
    pts["AQI_LABEL"] = [c[0] for c in classes]
    pts["AQI_COLOR"] = [c[1] for c in classes]

    # ---- Plot
    fig, ax = plt.subplots(figsize=FIG_SIZE, dpi=FIG_DPI)
    pasig_utm.boundary.plot(ax=ax, color="black", linewidth=0.8)
    pts.plot(ax=ax,
             color=pts["AQI_COLOR"],
             markersize=60,
             edgecolor="white", linewidth=0.8)

    ax.set_title(f"Stations by PM₂.₅ Category — {date_label}")
    ax.set_axis_off()

    # Legend (include 'No data' only if present)
    patches = [mpatches.Patch(color=c, label=l) for l, c in zip(AQI_LABELS, AQI_COLORS)]
    if (pts["AQI_LABEL"] == NO_DATA_LABEL).any():
        patches.append(mpatches.Patch(color=NO_DATA_COLOR, label=NO_DATA_LABEL))
    ax.legend(handles=patches, loc="lower left", bbox_to_anchor=(0.01,0.01),
              frameon=True, fontsize=9)

    out = os.path.join(OUT_DIR, f"{base}_aqi_points_local.png")
    plt.tight_layout(); plt.savefig(out, bbox_inches="tight"); plt.close(fig)
    print("Saved:", out)

Saved: C:\Users\krish\Desktop\SpatialCARE\Outputs\figures\aqi_points_local\date_2025-02-06_aqi_points_local.png
Saved: C:\Users\krish\Desktop\SpatialCARE\Outputs\figures\aqi_points_local\date_2025-02-07_aqi_points_local.png
Saved: C:\Users\krish\Desktop\SpatialCARE\Outputs\figures\aqi_points_local\date_2025-02-08_aqi_points_local.png
Saved: C:\Users\krish\Desktop\SpatialCARE\Outputs\figures\aqi_points_local\date_2025-02-09_aqi_points_local.png
Saved: C:\Users\krish\Desktop\SpatialCARE\Outputs\figures\aqi_points_local\date_2025-02-10_aqi_points_local.png
Saved: C:\Users\krish\Desktop\SpatialCARE\Outputs\figures\aqi_points_local\date_2025-02-11_aqi_points_local.png
Saved: C:\Users\krish\Desktop\SpatialCARE\Outputs\figures\aqi_points_local\date_2025-02-12_aqi_points_local.png
Saved: C:\Users\krish\Desktop\SpatialCARE\Outputs\figures\aqi_points_local\date_2025-02-13_aqi_points_local.png
Saved: C:\Users\krish\Desktop\SpatialCARE\Outputs\figures\aqi_points_local\date_2025-02-14_aqi_points_lo

# With Basemap + Roads

In [5]:
# 02_AQI_Point_Maps_PH_with_Roads.ipynb

# --- Imports & setup
import os, glob, warnings
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
from shapely.ops import unary_union
from fiona.env import Env
import fiona  # for safe layer removal in GPKG
import pandas as pd
import contextily as ctx  # for basemap tiles
from datetime import datetime

# --- Paths (updated)
DATA_DIR   = r"C:\Users\HP\Documents\SpatialCARE\Daily\DailyGPKG"
PASIG_SHP  = r"C:\Users\HP\Documents\PhD Class\Shapefile\MM\Pasig\Pasig.shp"
ROADS_SHP  = r"C:\Users\HP\Documents\PhD Class\Shapefile\MM\Pasig\PasigRN.shp"
OUT_DIR    = r"C:\Users\HP\Desktop\SpatialCARE\Outputs\figures\AQIMaps"
os.makedirs(OUT_DIR, exist_ok=True)

# --- CRS & figure
WGS84   = "+proj=longlat +datum=WGS84 +no_defs"
UTM51   = "+proj=utm +zone=51 +datum=WGS84 +units=m +no_defs"
WEB_MCT = "EPSG:3857"  # Web Mercator for basemaps
FIG_SIZE, FIG_DPI = (8.5, 8.5), 170

PM_CANDS  = ["pm25","PM25","PM_25","PM2_5","PM2.5"]
LON_CANDS = ["longitude","lon","LONG","x"]
LAT_CANDS = ["latitude","lat","LAT","y"]

def pick(cols, cands):
    for c in cands:
        if c in cols:
            return c
    return None

def dedupe_points(gdf):
    # de-duplicate by projected coords; returns UTM51
    t = gdf.to_crs(UTM51).copy()
    t["x"] = t.geometry.x; t["y"] = t.geometry.y
    return t.drop_duplicates(subset=["x","y"]).to_crs(UTM51)

def safe_write_gpkg(gdf, out_path, layer_name="layer"):
    """Write GeoDataFrame to GPKG, replacing the layer if it exists.
       Falls back to timestamped file if overwrite fails."""
    gdf = gdf.to_crs(WGS84)
    if os.path.exists(out_path):
        try:
            layers = fiona.listlayers(out_path)
            if layer_name in layers:
                fiona.remove(out_path, layer=layer_name, driver="GPKG")
        except Exception:
            base, ext = os.path.splitext(out_path)
            backup = f"{base}_backup_{datetime.now().strftime('%Y%m%d_%H%M%S')}{ext}"
            os.replace(out_path, backup)
            print(f"[info] Backed up existing GPKG to: {backup}")

    try:
        gdf.to_file(out_path, driver="GPKG", layer=layer_name)
        print(f"Filtered roads saved to: {out_path} (layer='{layer_name}')")
    except Exception as e:
        alt = out_path.replace(".gpkg", f"_{datetime.now().strftime('%Y%m%d_%H%M%S')}.gpkg")
        print(f"[warn] Primary write failed due to: {e}\n       Writing to new file: {alt}")
        gdf.to_file(alt, driver="GPKG", layer=layer_name)
        print(f"Filtered roads saved to: {alt} (layer='{layer_name}')")

# ==============================
# Philippine AQI for PM2.5 (24-hr) — DAO 2020-14
# ==============================
_PH_AQI_BANDS = [
    (0,   50),   # Good
    (51, 100),   # Fair
    (101,150),   # Unhealthy for Sensitive Groups
    (151,200),   # Very Unhealthy
    (201,300),   # Acutely Unhealthy
    (301,500),   # Emergency
]

# PM2.5 category breakpoints in µg/m³
_PH_PM25_BREAKS = [
    (0.0,   25.0),   # Good
    (25.1,  35.0),   # Fair
    (35.1,  45.0),   # USG
    (45.1,  55.0),   # Very Unhealthy
    (55.1,  90.0),   # Acutely Unhealthy
    (91.0,  np.inf), # Emergency
]

AQI_PH_LABELS = [
    "Good (0–50)",
    "Fair (51–100)",
    "Unhealthy for sensitive groups (101–150)",
    "Very unhealthy (151–200)",
    "Acutely unhealthy (201–300)",
    "Emergency (301–500)",
]
AQI_PH_COLORS = [
    "#00E400",  # Green
    "#FFFF00",  # Yellow
    "#FF7E00",  # Orange
    "#FF0000",  # Red
    "#8F3F97",  # Purple
    "#7E0023",  # Maroon
]

NO_DATA_LABEL = "No data"
NO_DATA_COLOR = "#bdbdbd"

def _trunc_to_0p1(x):
    return np.floor(x * 10.0) / 10.0

def pm25_to_ph_aqi(x):
    v = pd.to_numeric(x, errors="coerce")
    if not np.isfinite(v):
        return np.nan
    v = max(0.0, float(v))
    v = _trunc_to_0p1(v)
    for (BPLo, BPHi), (ILo, IHi) in zip(_PH_PM25_BREAKS, _PH_AQI_BANDS):
        if BPLo <= v <= BPHi or (np.isinf(BPHi) and v >= BPLo):
            if np.isfinite(BPHi):
                return (IHi - ILo) / (BPHi - BPLo) * (v - BPLo) + ILo
            # Emergency band (open-ended): interpolate to 500 with a practical ceiling
            ceiling = 150.0
            vv = min(v, ceiling)
            Ip = (500 - 301) / (ceiling - 91.0) * (vv - 91.0) + 301
            return min(500.0, Ip)
    return np.nan

def aqi_to_ph_category_color(aqi):
    if not np.isfinite(aqi):
        return NO_DATA_LABEL, NO_DATA_COLOR
    aqi = max(0.0, float(aqi))
    if aqi <= 50:    return AQI_PH_LABELS[0], AQI_PH_COLORS[0]
    if aqi <= 100:   return AQI_PH_LABELS[1], AQI_PH_COLORS[1]
    if aqi <= 150:   return AQI_PH_LABELS[2], AQI_PH_COLORS[2]
    if aqi <= 200:   return AQI_PH_LABELS[3], AQI_PH_COLORS[3]
    if aqi <= 300:   return AQI_PH_LABELS[4], AQI_PH_COLORS[4]
    return AQI_PH_LABELS[5], AQI_PH_COLORS[5]

# --- Read Pasig boundary
with Env(SHAPE_RESTORE_SHX="YES"):
    pasig = gpd.read_file(PASIG_SHP)
if pasig.crs is None:
    pasig = pasig.set_crs(WGS84)
pasig_utm = pasig.to_crs(UTM51)

# --- Read & filter road network (primary/secondary/tertiary)
with Env(SHAPE_RESTORE_SHX="YES"):
    roads = gpd.read_file(ROADS_SHP)
if roads.crs is None:
    roads = roads.set_crs(WGS84)

# choose attribute column for road class
col = None
for candidate in ("highway", "fclass", "type"):
    if candidate in roads.columns:
        col = candidate
        break
if col is None:
    raise SystemExit("No 'highway'/'fclass'/'type' column found in road shapefile.")

wanted = {"primary", "secondary", "tertiary"}
roads[col] = roads[col].astype(str).str.lower()
roads_sel = roads[roads[col].isin(wanted)].copy()

# project & clip to Pasig boundary (UTM for geometry ops)
roads_sel_utm = roads_sel.to_crs(UTM51)
try:
    roads_clip = gpd.clip(roads_sel_utm, pasig_utm)
except Exception:
    roads_clip = gpd.overlay(roads_sel_utm, pasig_utm[["geometry"]], how="intersection")

# --- Gather stations per day and generate maps
files = sorted(glob.glob(os.path.join(DATA_DIR, "date_2025-*.gpkg")))
if not files:
    raise SystemExit("No daily GPKG files found.")

for f in files:
    base = os.path.splitext(os.path.basename(f))[0]
    date_label = base.replace("date_","")

    g = gpd.read_file(f)
    if g.crs is None:
        g = g.set_crs(WGS84)

    # Ensure geometry exists (build from lon/lat if necessary)
    if ("geometry" not in g) or g.geometry.is_empty.all():
        lon = pick(g.columns, LON_CANDS)
        lat = pick(g.columns, LAT_CANDS)
        if lon and lat:
            g = g.set_geometry(gpd.points_from_xy(g[lon], g[lat], crs=g.crs))
        else:
            print(f"Skip (no geometry & no lon/lat): {base}")
            continue

    pm_col = pick(g.columns, PM_CANDS)
    if pm_col is None:
        print(f"Skip (no PM2.5 column): {base}")
        continue

    # --- Compute PH AQI & style
    pts_utm = g.to_crs(UTM51).copy()
    pts_utm["AQI_PH"] = pts_utm[pm_col].apply(pm25_to_ph_aqi).round(0)
    cats = pts_utm["AQI_PH"].apply(aqi_to_ph_category_color)
    pts_utm["AQI_LABEL"] = [c[0] for c in cats]
    pts_utm["AQI_COLOR"] = [c[1] for c in cats]

    # --- Reproject display layers to Web Mercator (for basemap)
    pasig_web = pasig_utm.to_crs(WEB_MCT)
    roads_web = roads_clip.to_crs(WEB_MCT)
    pts_web   = pts_utm.to_crs(WEB_MCT)

    # --- Plot
    fig, ax = plt.subplots(figsize=FIG_SIZE, dpi=FIG_DPI)

    # 1) Boundary and roads (so extent is set)
    pasig_web.boundary.plot(ax=ax, linewidth=2, color="#ffffff", zorder=2)

    # Roads by class (distinct styles for legend)
    road_styles = {
        "primary":   dict(linewidth=2.0, color="#d62728", zorder=3),  # red
        "secondary": dict(linewidth=1.6, color="#1f77b4", zorder=3),  # blue
        "tertiary":  dict(linewidth=1.2, color="#2ca02c", zorder=3),  # green
    }
    for klass, sty in road_styles.items():
        sub = roads_web.loc[roads_web[col] == klass]
        if not sub.empty:
            sub.plot(ax=ax, **sty)

    # Stations colored by PH AQI
    pts_web.plot(
        ax=ax,
        color=pts_web["AQI_COLOR"],
        markersize=65,
        marker="o",
        edgecolor="white",
        linewidth=0.9,
        zorder=4
    )

    # 2) Fix extent using Pasig bounds (with padding)
    minx, miny, maxx, maxy = pasig_web.total_bounds
    pad_x = (maxx - minx) * 0.03
    pad_y = (maxy - miny) * 0.03
    ax.set_xlim(minx - pad_x, maxx + pad_x)
    ax.set_ylim(miny - pad_y, maxy + pad_y)

    # 3) Basemap behind vectors
    ctx.add_basemap(
        ax,
        source=ctx.providers.Esri.WorldImagery,
        crs=pasig_web.crs,
        zoom=14,
        reset_extent=False,
        zorder=0
    )

    # --- Legends: combine AQI category patches + road line handles
    aqi_handles = [Patch(facecolor=c, edgecolor='none', label=l) for l, c in zip(AQI_PH_LABELS, AQI_PH_COLORS)]
    if (pts_web["AQI_LABEL"] == NO_DATA_LABEL).any():
        aqi_handles.append(Patch(facecolor=NO_DATA_COLOR, edgecolor='none', label=NO_DATA_LABEL))

    road_handles = [
        Line2D([0], [0], color=road_styles["primary"]["color"],   lw=road_styles["primary"]["linewidth"],   label="Primary road"),
        Line2D([0], [0], color=road_styles["secondary"]["color"], lw=road_styles["secondary"]["linewidth"], label="Secondary road"),
        Line2D([0], [0], color=road_styles["tertiary"]["color"],  lw=road_styles["tertiary"]["linewidth"],  label="Tertiary road"),
    ]

    # Create two side-by-side legends (roads + AQI)
    leg1 = ax.legend(handles=road_handles, title="Road classes", frameon=True, loc="upper left",
                     bbox_to_anchor=(1.02, 1.00), borderaxespad=0)
    ax.add_artist(leg1)
    leg2 = ax.legend(handles=aqi_handles, title="PH AQI (PM₂.₅, 24-hr)", frameon=True, loc="upper left",
                     bbox_to_anchor=(1.02, 0.42), borderaxespad=0)

    # Title and cosmetics
    ax.set_title(f"Pasig — PH AQI (PM₂.₅, 24-hr) with Primary/Secondary/Tertiary Roads — {date_label}")
    ax.set_axis_off()

    out = os.path.join(OUT_DIR, f"{base}_PH_AQI_with_roads.png")
    plt.tight_layout()
    plt.savefig(out, bbox_inches="tight")
    plt.close(fig)
    print("Saved figure:", out)

# --- Save filtered roads safely to GPKG (WGS84), if you want the clipped network persisted
roads_out   = os.path.join(OUT_DIR, "Pasig_Roads_PriSecTer.gpkg")
layer_name  = "Pasig_Roads_PriSecTer"
safe_write_gpkg(roads_clip, roads_out, layer_name=layer_name)


Saved figure: C:\Users\HP\Desktop\SpatialCARE\Outputs\figures\AQIMaps\date_2025-02-06_PH_AQI_with_roads.png
Saved figure: C:\Users\HP\Desktop\SpatialCARE\Outputs\figures\AQIMaps\date_2025-02-07_PH_AQI_with_roads.png
Saved figure: C:\Users\HP\Desktop\SpatialCARE\Outputs\figures\AQIMaps\date_2025-02-08_PH_AQI_with_roads.png
Saved figure: C:\Users\HP\Desktop\SpatialCARE\Outputs\figures\AQIMaps\date_2025-02-09_PH_AQI_with_roads.png
Saved figure: C:\Users\HP\Desktop\SpatialCARE\Outputs\figures\AQIMaps\date_2025-02-10_PH_AQI_with_roads.png
Saved figure: C:\Users\HP\Desktop\SpatialCARE\Outputs\figures\AQIMaps\date_2025-02-11_PH_AQI_with_roads.png
Saved figure: C:\Users\HP\Desktop\SpatialCARE\Outputs\figures\AQIMaps\date_2025-02-12_PH_AQI_with_roads.png
Saved figure: C:\Users\HP\Desktop\SpatialCARE\Outputs\figures\AQIMaps\date_2025-02-13_PH_AQI_with_roads.png
Saved figure: C:\Users\HP\Desktop\SpatialCARE\Outputs\figures\AQIMaps\date_2025-02-14_PH_AQI_with_roads.png
Saved figure: C:\Users\HP\De

In [1]:
import os
import glob
from PIL import Image

# Folder where your PNG files are located
img_dir = r"D:\Work\SpatialCARE\Outputs\Daily\figures\AQIMaps"

# Get all .png files sorted by filename (important for order)
png_files = sorted(glob.glob(os.path.join(img_dir, "*.png")))

# Load images
frames = [Image.open(f) for f in png_files]

# Output file
gif_path = os.path.join(img_dir, "AQIMaps_Animation.gif")

# Save as GIF
frames[0].save(
    gif_path,
    save_all=True,
    append_images=frames[1:],
    duration=500,   # duration per frame in milliseconds (adjust speed here)
    loop=0          # 0 means infinite loop
)

print(f"GIF saved at: {gif_path}")

GIF saved at: D:\Work\SpatialCARE\Outputs\Daily\figures\AQIMaps\AQIMaps_Animation.gif
